In [105]:
from sklearn.linear_model import  LogisticRegression
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
import pandas as pd
import os

In [106]:
os.listdir()

['.DS_Store',
 '.ipynb_checkpoints',
 'Practise.ipynb',
 'RegularSeasonDetailedResults.csv',
 'Seasons.csv',
 'TourneyCompactResults.csv',
 'TourneySeeds.csv']

In [107]:
df_seeds=pd.read_csv('TourneySeeds.csv')
df_tour=pd.read_csv('TourneyCompactResults.csv')

In [108]:
df_seeds.head()

,Season,Seed,Team
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [109]:
df_tour.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [112]:
#drop columns which we are not planning on using
df_tour.drop(labels=['Daynum','Wscore','Lscore','Wloc','Numot'],inplace=True,axis=1)

In [113]:
df_tour.head()

,Season,Wteam,Lteam
0,1985,1116,1234
1,1985,1120,1345
2,1985,1207,1250
3,1985,1229,1425
4,1985,1242,1325


In [114]:
def seed_to_int(seed):
    '''Just get the digits from the seeding ,return as int'''
    s_int=int(seed[1:3])
    return(s_int)

In [115]:
df_seeds['n_seed']=df_seeds.Seed.apply(func=seed_to_int)

In [118]:
df_winseeds = df_seeds.rename(columns={'Team':'Wteam', 'n_seed':'win_seed'})
df_lossseeds = df_seeds.rename(columns={'Team':'Lteam', 'n_seed':'loss_seed'})

In [120]:
df_dummy=pd.merge(left=df_tour,right=df_winseeds,how='left',on=['Season','Wteam'])
df_concat=pd.merge(left=df_dummy,right=df_lossseeds,how='left',on=['Season','Lteam'])
df_concat['seed_diff']=df_concat.win_seed-df_concat.loss_seed

In [121]:
df_concat.head()

,Season,Wteam,Lteam,Seed_x,win_seed,Seed_y,loss_seed,seed_diff
0,1985,1116,1234,X09,9,X08,8,1
1,1985,1120,1345,Z11,11,Z06,6,5
2,1985,1207,1250,W01,1,W16,16,-15
3,1985,1229,1425,Y09,9,Y08,8,1
4,1985,1242,1325,Z03,3,Z14,14,-11


In [125]:
df_wins=pd.DataFrame()
df_wins['seed_diff']=df_concat['seed_diff']
df_wins['result']=1
df_losses=pd.DataFrame()
df_losses['seed_diff']=-df_concat['seed_diff']
df_losses['result']=0
df_for_prediction=pd.concat((df_wins,df_losses),axis=0)

In [139]:
X_train=df_for_prediction.seed_diff.values.reshape(-1,1)
y_train=df_for_predictions.result.values
X_train,y_train=shuffle(X_train,y_train)

In [143]:
logreg=LogisticRegression()
params={'C':np.logspace(start=-5,stop=3,num=9)}
logreg=GridSearchCV(logreg,param_grid=params,scoring='neg_log_loss',refit=True)
clf.fit(X_train,y_train)
print('Best score is {0},with best C {1}'.format(clf.best_score_,clf.best_params_['C']))

Best score is -0.5536943752612554,with best C 0.01


In [144]:
os.listdir()

['.DS_Store',
 '.ipynb_checkpoints',
 'Practise.ipynb',
 'RegularSeasonDetailedResults.csv',
 'SampleSubmission.csv',
 'Seasons.csv',
 'TourneyCompactResults.csv',
 'TourneySeeds.csv']

In [146]:
#get prediction data
df_pred=pd.read_csv('SampleSubmission.csv')

In [151]:
n_test=len(df_pred)

In [150]:
def get_year_t1_t2(id):
    return(int(x) for x in id.split('_'))

In [154]:
df_test=np.zeros(shape=(n_test,1))
for index,row in df_pred.iterrows():
    year,t1,t2=get_year_t1_t2(row['Id'])
    t1_seed=df_seeds[(df_seeds.Team==t1) & (df_seeds.Season==year)].n_seed.values[0]
    t2_seed=df_seeds[(df_seeds.Team==t2) & (df_seeds.Season==year)].n_seed.values[0]
    diff_seed=t1_seed-t2_seed
    df_test[index,0]=diff_seed

In [171]:
preds=clf.predict_proba(df_test)[:,1]
preds=np.clip(preds,0,1)
df_pred.Pred=preds

In [174]:
df_pred.to_csv('Result.csv')